In [ ]:
# !huggingface-cli login --token  
!pip install datasets > -
!pip install rouge_score > -
!pip install flash_attn
!pip install transtormers
!pip install --upgrade transformers
!pip install accelerate
!pip install peft
!pip install bitsandbytes
!pip install -qqq trl

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
ERROR: Could not find a version that satisfies the requirement transtormers (from versions: none)
ERROR: No matching distribution found for transtormers


In [2]:
! apt update && apt install git -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,390 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,671 kB]
Get:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,676 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,059 kB]
Get:13 http://archive.ubuntu.com

In [3]:
from datasets import Dataset, load_dataset
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for Salesforce/dialogstudio contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Salesforce/dialogstudio
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datas

In [ ]:
import json
import pandas as pd
import re

def generate_prompt(conversation: str) -> str:
    return f"""### Instruction:\nBelow is a conversation between a human and an agent. Write a summary of the conversation.

### Input:
{conversation.strip()}
""".strip()

def cleaning(text):

    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"http\S+", "", text)

    return re.sub(r"\^[^ ]+", "", text)


def conversation_extraction(data_point):
    text = ""
    for item in data_point["log"]:
        customer = cleaning(item["user utterance"])
        text += f"Customer: {customer.strip()}\n"

        agent = cleaning(item["system response"])
        text += f"Agent: {agent.strip()}\n"

    return text



def redesign_data(data_point):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)

    conversation_text = conversation_extraction(data_point)
    text = "### Instruction:\nBelow is a conversation between a human and an agent. Write a summary of the conversation.\n\n### Input:\n" + conversation_text +  "\n\n### Output:\n" + summary

    return {
        "conversation": conversation_text,
        "summary": summary,
        "text": text,
        "prompt": generate_prompt(conversation_text),
    }


def preprocess_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(redesign_data)
        .remove_columns(
            [
                "original dialog id","new dialog id","dialog index",
                "original dialog info","log","prompt",
            ]
        )
    )

In [ ]:
dataset["train"] = preprocess_dataset(dataset["train"])
dataset["validation"] = preprocess_dataset(dataset["validation"])
dataset["test"] = preprocess_dataset(dataset["test"])

In [ ]:
from datasets import load_metric
from tqdm import tqdm
rouge = load_metric('rouge')
example_indices = [109]
line = "---------------------------------------------------------------------------------------------------"

<ipython-input-6-f3aa68319492>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def make_prompt(examples, sample):
    prompt = ''
    for index in examples:
        conversation = dataset['test'][index]['conversation']
        summary = dataset['test'][index]['summary']
        prompt += f"""Below is a conversation between a human and an AI agent. With a summary of the conversation.
### Input:
{conversation}
### Output:
{summary}

"""

    conversation = dataset['test'][sample]['conversation']

    prompt += f"""Below is a conversation between a human and an AI agent. Write a summary of the conversation.
### Input:
{conversation}
### Output:
"""
    return prompt
prompt = print(make_prompt([30], 109))

Below is a conversation between a human and an AI agent. With a summary of the conversation.
### Input:
Customer: On went to buy a Lip na mpesa wrist band in your shop in Thika and was sent to Annas Mall where i dint find your Team!
Agent: Apologies for this, check with any M-PESA Agent, or Safaricom authorized dealer. You may also go back to our shop in Thika -(contd)to verify whether we have restocked.
Customer: I was at your shop at Thika and they told me they were not sure where the guys dealing with the wrist bands were
Agent: The mpesa 1 tap is not available at Thika its being piloted in Nairobi, Mombasa, Kisumu, Eldoret and Nyeri towns ... cont... currently. To get it you may visit the nearest town which is Nairobi to purchase one at a cost of 20/-.
Customer: Would you please confirm for me if I will get the wrist band at your shop in Moi Avenue today so I don't have to travel for nothing.
Agent: Hi, yes you will and at the moment it can only be used in the towns mentioned in ou

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel, get_peft_model, TaskType
from trl import SFTTrainer
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-3b-4e1t")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    "stabilityai/stablelm-3b-4e1t",
    use_safetensors=True,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import numpy as np

# Function to calculate model size
def calculate_model_size(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params / (1024 ** 2)  # Convert to megabytes


model_size_after_quantization = calculate_model_size(model)


In [5]:
model_size_after_quantization

245.9423828125

In [6]:
model2 = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-3b-4e1t", trust_remote_code=True,torch_dtype="auto")

In [7]:
model_size_before_quantization = calculate_model_size(model2)
model_size_reduction = model_size_before_quantization - model_size_after_quantization

print(f"Model size before quantization: {model_size_before_quantization} MB")
print(f"Model size after quantization: {model_size_after_quantization} MB")
print(f"Model size reduction after quantization: {model_size_reduction} MB")

Model size before quantization: 2665.9423828125 MB
Model size after quantization: 245.9423828125 MB
Model size reduction after quantization: 2420.0 MB


In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=32,
    lora_alpha=8,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ],
    bias="none"
)

model.config.use_cache = False
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 2,816,414,720 || trainable%: 0.7446176108609459


In [ ]:
training_arguments = TrainingArguments(
    output_dir="lora-r32-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_steps=1,
    num_train_epochs=5,
    push_to_hub=False
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length= 4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.model.print_trainable_parameters()
tokenizer.pad_token = tokenizer.eos_token

trainable params: 20,971,520 || all params: 2,816,414,720 || trainable%: 0.7446176108609459


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.677700,2.075731
2,1.935600,2.058427
3,1.299700,2.063561
4,1.845900,2.091055
5,1.658000,2.105810


TrainOutput(global_step=4395, training_loss=1.9214049900486743, metrics={'train_runtime': 2150.5955, 'train_samples_per_second': 2.044, 'train_steps_per_second': 2.044, 'total_flos': 2.28768123250176e+16, 'train_loss': 1.9214049900486743, 'epoch': 5.0})

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stablelm-3b-4e1t",
  trust_remote_code=True,
  torch_dtype="auto",
  use_safetensors=True,
).to("cuda")
peft_model = PeftModel.from_pretrained(model, "lora-r32-finetuned/checkpoint-4395", from_transformers=True)
model = peft_model.merge_and_unload()

model.config.pad_token_id = tokenizer.eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id

In [ ]:
results_zero_shot = []
results_one_shot = []
for index in tqdm(range(110)):
    model.config.pad_token_id = tokenizer.eos_token_id
    summary = dataset['test'][index]['summary']
    if 109 > index:
      prompt = make_prompt([index+1], index)
    else:
      prompt = make_prompt([30], index)
    inputs = tokenizer(prompt, return_tensors='pt').to("cuda")
    token = model.generate(
        **inputs,
        max_new_tokens=64,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id,
        top_p=0.95,
        do_sample=True,
        )
    completion_tokens = token[0][inputs['input_ids'].size(1):]
    completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)
    results_one_shot.append(rouge.compute(predictions=[completion], references=[summary]))
    prompt = dataset['test'][index]['text']
    inputs = tokenizer(prompt, return_tensors='pt').to("cuda")
    token = model.generate(
        **inputs,
        max_new_tokens=64,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id,
        top_p=0.95,
        do_sample=True,
        )
    completion_tokens = token[0][inputs['input_ids'].size(1):]
    completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)
    results_zero_shot.append(rouge.compute(predictions=[completion], references=[summary]))
average_score_zero_shot = {metric: sum([result[metric].mid.fmeasure for result in results_zero_shot]) / len(results_zero_shot)
                  for metric in results_zero_shot[0]}
average_score_one_shot = {metric: sum([result[metric].mid.fmeasure for result in results_one_shot]) / len(results_one_shot)
                  for metric in results_one_shot[0]}
print(f"average score one shot:{average_score_one_shot}\naverage score zero shot:{average_score_zero_shot}")

100%|██████████| 110/110 [11:19<00:00,  6.18s/it]

average score one shot:{'rouge1': 0.3681059483218816, 'rouge2': 0.13981877957788952, 'rougeL': 0.2953305409730834, 'rougeLsum': 0.30379438020703853}
average score zero shot:{'rouge1': 0.4332597143242216, 'rouge2': 0.25816331149057714, 'rougeL': 0.3617806393058581, 'rougeLsum': 0.3747735275398468}


In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): StableLMEpochForCausalLM(
      (model): StableLMEpochModel(
        (embed_tokens): Embedding(50304, 2560)
        (layers): ModuleList(
          (0-31): 32 x DecoderLayer(
            (self_attn): Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=25